In [1]:
!pip install hyperopt

     ---------------------------------------- 1.6/1.6 MB 11.2 MB/s eta 0:00:00
     -------------------------------------- 200.5/200.5 kB 6.1 MB/s eta 0:00:00


### HyperOpt 라이브러리의 주요 함수와 개념

- fmin: 이 함수는 최적화 과정을 수행하는 주요 함수입니다. 이 함수는 목표 함수(fn), 검색 공간(space), 최적화 알고리즘(algo), 그리고 최대 평가 횟수(max_evals) 등을 인자로 받습니다. 이 함수는 최적의 하이퍼파라미터 설정을 찾기 위해 주어진 알고리즘을 사용하여 검색 공간을 탐색하고, 각 설정에 대해 목표 함수를 평가합니다.

- tpe: 이것은 Tree-structured Parzen Estimator(TPE) 알고리즘을 나타내며, HyperOpt에서 사용할 수 있는 최적화 알고리즘 중 하나입니다. TPE는 베이지안 최적화 알고리즘의 한 종류로, 이전의 평가 결과를 기반으로 하여 다음에 테스트할 하이퍼파라미터를 선택합니다. 이렇게 하면 더 좋은 결과를 내는 하이퍼파라미터의 영역을 더 자주 탐색하게 됩니다.

- hp: 이것은 HyperOpt의 검색 공간을 정의하는 데 사용되는 함수들을 담고 있는 모듈입니다. 예를 들어, hp.uniform('x', 0, 1)은 0과 1 사이에서 균일하게 분포한 실수 값을 가지는 'x'라는 하이퍼파라미터를 정의합니다. 다른 함수로는 hp.choice (여러 선택 사항 중 하나를 선택), hp.normal (정규 분포에서 값을 선택) 등이 있습니다.

- STATUS_OK: 이것은 목표 함수가 성공적으로 완료되었음을 나타내는 상태 코드입니다. 목표 함수는 손실값과 상태 코드를 포함하는 딕셔너리를 반환해야 합니다. 이 상태 코드는 HyperOpt가 평가를 제대로 수행했는지 판단하는 데 사용됩니다. 만약 다른 값을 반환하면, HyperOpt는 해당 평가가 실패했다고 판단하고, 해당 설정을 더 이상 탐색하지 않습니다.

### Tree-structured Parzen Estimator (TPE)
- 하이퍼파라미터 최적화를 위한 알고리즘 중 하나입니다. Bayesian optimization의 한 종류로, 하이퍼파라미터의 새로운 조합을 선택하는 방법으로 주로 사용됩니다.

- TPE 알고리즘의 핵심 아이디어는 과거의 평가 결과를 기반으로 하이퍼파라미터의 좋은 값과 나쁜 값에 대한 확률 모델을 구축하고, 이 모델을 사용하여 다음에 시도할 하이퍼파라미터 값을 선택하는 것입니다.

- TPE는 이를 위해 두 개의 확률 분포 l(x)와 g(x)를 사용합니다. 여기서 x는 하이퍼파라미터를 의미하며, l(x)는 과거의 좋은 하이퍼파라미터 값들에 대한 분포를 나타내고, g(x)는 모든 과거의 하이퍼파라미터 값들에 대한 분포를 나타냅니다.

- 이 두 분포를 이용해 새로운 하이퍼파라미터 값 x를 선택할 때, l(x)/g(x)가 큰 값, 즉 과거의 좋은 하이퍼파라미터 값들에 더 가까운 값을 선택합니다.

- 이런 방식으로, TPE는 점점 더 좋은 하이퍼파라미터 값을 찾아가는 탐색 과정을 수행합니다. 이는 전통적인 Grid Search나 Random Search에 비해 효율적이며, 보다 적은 평가로 좋은 하이퍼파라미터 값을 찾을 수 있게 합니다.

### make_classification 함수 매개변수

- n_samples: 생성할 샘플의 개수를 지정합니다. 기본값은 100입니다.
- n_features: 독립 변수의 개수를 지정합니다. 기본값은 20입니다.
- n_informative: 종속 변수와 관련된 독립 변수의 수를 지정합니다. 기본값은 2입니다.
- n_redundant: 다른 독립 변수의 선형 조합으로 생성된 독립 변수의 수를 지정합니다. 기본값은 2입니다.
- n_classes: 생성할 클래스(또는 레이블)의 개수를 지정합니다. 기본값은 2입니다.

이 함수는 X와 y의 두 개의 배열을 반환합니다. X는 n_samples x n_features 크기의 2차원 배열로, 독립 변수를 포함하고 있습니다. y는 n_samples 크기의 1차원 배열로, 각 샘플의 클래스 레이블을 포함하고 있습니다.

In [2]:
# 1000개의 샘플, 10개의 독립 변수, 그리고 2개의 클래스로 구성된 가상 데이터셋을 생성 
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=1000, n_features=10, n_classes=2)

In [6]:
# make_classification 함수는 sklearn.datasets 모듈의 일부로, 분류 문제에 사용할 수 있는 
# 가상 데이터셋을 생성하는 데 사용

# 데이터 생성
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp, STATUS_OK

X, y = make_classification()

# Hyper opt 에 의해 최적화될 목표 함수를 정의합니다.
# 이 함수는 손실값을 반환하며, 손실값이 최소가 되는 하이퍼파라미터를 찾는 것이 목표입니다. 
def objective(args):
    # 모델 설정
    # C는 규제의 역수로, C값이 작을수록 규제가 강하며 C값이 클수록 규제가 약해집니다.
    # Logistic Regression 모델을 초기화하고, Hyper opt 가 최적화할 하이퍼파라미터 C를 설정합니다. 
    model = LogisticRegression(C=args['C'])

    # 모델 검증
    # 5-fold 교차 검증을 수행하고, 평균 정확도를 계산 
    score = cross_val_score(model, X, y, cv=5).mean()
    
    
    # 손실값 계산(분류 문제에서는 정확도를 사용하므로 1 정확도를 반환) 
    # 분류 문제에서는 정확도를 사용하므로, 손실값을 1- 정확도로 계산 
    loss = 1 - score

    # 손실값과 상태 코드를 반환합니다.
    return {'loss': loss, 'status': STATUS_OK}

# 검색 공간 정의
# 최적화할 하이퍼파라미터인 C의 범위를 정의합니다.
space = {
    'C': hp.loguniform('C', -5, 0)
}

# 최적화 실행
# TPE 알고리즘을 사용하여 목표 함수를 최적화하고,
# 최적의 하이퍼파라미터를 찾습니다. 최대 100회의 평가를 수행합니다. 

best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100)
print(best)
    
    

100%|██████████████████████████████████████████████| 100/100 [00:01<00:00, 54.20trial/s, best loss: 0.1200000000000001]
{'C': 0.007855075583061526}


In [7]:
from hyperopt import hp
# -10 ~ 10까지 1간격을 가지는 입력 변수 x와 -15 ~ 15까지 1간격으로 입력 변수 y 설정. 
# hp.quniform(label, low, high, q) 함수는 low와 high 사이에서 일정 간격 q

search_space = {'x': hp.quniform('x', -10, 10, 1), 'y': hp.quniform('y', -15, 15, 1) }


In [8]:
from hyperopt import STATUS_OK

# 목적 함수를 생성. 변숫값과 변수 검색 공간을 가지는 딕셔너리를 인자로 받고, 특정 값을 반환 
# search_space는 하이퍼파라미터의 조합을 나타내는 딕셔너리입니다. 이 딕셔너리는 'x'와 'y'라는 
# 두 개의 키를 가지고 있습니다. 이 두 키는 각각 목표 함수에서 사용될 두 개의 변수를 나타냅니다. 
def objective_func(search_space):
    x =search_space['x']
    y = search_space['y']
    retval = x**2 - 20*y # retval 이라는 변수에 목표 함수의 식을 계산한 결과를 저장

    return retval

In [10]:
from hyperopt import fmin, tpe, hp, Trials
import numpy as np

# 입력 결괏값을 저장한 Trials 객체값 생성.
trial_val = Trials()

# 목적 함수의 최솟값을 반환하는 최적 입력 변수값을 5번의 입력값 시도 (max_evals=5)로 찾아냄. 
best_01 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=5,
               trials=trial_val, rstate=np.random.default_rng(seed=0))

print('best', best_01)


100%|█████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 614.66trial/s, best loss: -224.0]
best {'x': -4.0, 'y': 12.0}


In [11]:
from hyperopt import fmin, tpe, hp, Trials
import numpy as np

# 입력 결괏값을 저장한 Trials 객체값 생성.
trial_val = Trials()

# 목적 함수의 최솟값을 반환하는 최적 입력 변수값을 5번의 입력값 시도 (max_evals=20)로 찾아냄. 
best_01 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=20,
               trials=trial_val, rstate=np.random.default_rng(seed=0))

print('best', best_01)

100%|███████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 689.84trial/s, best loss: -296.0]
best {'x': 2.0, 'y': 15.0}


In [13]:
# fmin( )에 인자로 들어가는 Trials 객체의 result 속성에 파이썬 리스트로 목적 함수 반환값들이 저장됨 
# 리스트 내부의 개별 원소는 {'loss': 함수 반환값, 'status': 반환 상태값} 와 같은 딕셔너리임.

print(trial_val.results)

[{'loss': -64.0, 'status': 'ok'}, {'loss': -184.0, 'status': 'ok'}, {'loss': 56.0, 'status': 'ok'}, {'loss': -224.0, 'status': 'ok'}, {'loss': 61.0, 'status': 'ok'}, {'loss': -296.0, 'status': 'ok'}, {'loss': -40.0, 'status': 'ok'}, {'loss': 281.0, 'status': 'ok'}, {'loss': 64.0, 'status': 'ok'}, {'loss': 100.0, 'status': 'ok'}, {'loss': 60.0, 'status': 'ok'}, {'loss': -39.0, 'status': 'ok'}, {'loss': 1.0, 'status': 'ok'}, {'loss': -164.0, 'status': 'ok'}, {'loss': 21.0, 'status': 'ok'}, {'loss': -56.0, 'status': 'ok'}, {'loss': 284.0, 'status': 'ok'}, {'loss': 176.0, 'status': 'ok'}, {'loss': -171.0, 'status': 'ok'}, {'loss': 0.0, 'status': 'ok'}]


In [14]:
# Trials 객체의 vals 속성에 {'입력변수명': 개별 수행 시마다 입력된 값 리스트} 형태로 저장됨. 
print(trial_val.vals)

{'x': [-6.0, -4.0, 4.0, -4.0, 9.0, 2.0, 10.0, -9.0, -8.0, -0.0, -0.0, 1.0, 9.0, 6.0, 9.0, 2.0, -2.0, -4.0, 7.0, -0.0], 'y': [5.0, 10.0, -2.0, 12.0, 1.0, 15.0, 7.0, -10.0, 0.0, -5.0, -3.0, 2.0, 4.0, 10.0, 3.0, 3.0, -14.0, -8.0, 11.0, -0.0]}


In [18]:
import pandas as pd

# results에서 loss 키 값에 해당하는 벨류들을 추출하여 list로 생성
losses = [loss_dict['loss'] for loss_dict in trial_val.results]

# DataFrame으로 생성.
result_df = pd.DataFrame({'x':trial_val.vals['x'], 'y': trial_val.vals['y'], 'losses' : losses})
result_df

,x,y,losses
0,-6.0,5.0,-64.0
1,-4.0,10.0,-184.0
2,4.0,-2.0,56.0
3,-4.0,12.0,-224.0
4,9.0,1.0,61.0
5,2.0,15.0,-296.0
6,10.0,7.0,-40.0
7,-9.0,-10.0,281.0
8,-8.0,0.0,64.0
9,-0.0,-5.0,100.0


과제2_0523. 위스콘신 유방암 데이터 세트를 로딩해서 XGBooster 알고리즘으로 모델링 및 평가를 수행하세요. 단, HyperOpt를 이용해 하이퍼 파라미터를 최적화하세요

1. 데이터 다운로드
: 데이터 전처리 후

2. 학습, 검증, 평가 데이터 분리

3. 검색 공간 설정 for HP
: xgb_search_space

4. 목적함수 설정

5. 최적 하이퍼 파라미터 산출

6. 평가 함수 설정

7. 학습

8. 평가

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

dataset = load_breast_cancer()    # 데이터 불러와서

cancer_df = pd.DataFrame(data=dataset.data, columns=dataset.feature_names)   # 데이터프레임으로 만듦
cancer_df['target']= dataset.target 
X_features = cancer_df.iloc[:, :-1] #
y_label = cancer_df.iloc[:, -1]


In [3]:
# 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
X_train, X_test, y_train, y_test=train_test_split(X_features, y_label, test_size=0.2, random_state=156 )

# 앞에서 추출한 학습 데이터를 다시 학습과 검증 데이터로 분리
X_tr, X_val, y_tr, y_val= train_test_split(X_train, y_train, test_size=0.1, random_state=156 )


In [4]:
from hyperopt import hp

# 하이퍼 파라미터의 검색 공간 설정
# max_depth는 5에서 20까지 1간격으로, min_child_weight는 1에서 2까지 1간격으로
# colsample_bytree는 0.5에서 1사이, learning_rate는 0.01에서 0.2 사이 정규 분포된 값으로 검색.
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 20, 1), 
                    'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),   # 무작위 선택 피처 비율
                   }

# 이 범위 내에서 하이퍼 파라미터를 선정해서 

In [5]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from hyperopt import STATUS_OK

# 그 성과를 측정하는 것이 목적 함수
# fmin()에서 입력된 search_space 값으로 입력된 모든 값은 실수형임.
# XGBClassifier의 정수형 하이퍼 파라미터는 정수형 변환을 해줘야 함.
# 정확도는 높을수록 더 좋은 수치임. -1 * 정확도를 곱해서 큰 정확도 값일수록 최소가 되도록 변환
def objective_func(search_space):
    # 수행 시간 절약을 위해 nestimators는 100으로 축소
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            learning_rate=search_space['learning_rate'],
                            colsample_bytree=search_space['colsample_bytree'],
                            eval_metric='logloss')
    accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3)   
    # accuracy는 높은게 좋은게 fmin은 최소값을 구하기 때문에 아래서 -1를 곱한 것 
    
    # accuracy는 cv=3 개수만큼 roc-auc 결과를 리스트로 가짐. 이를 평균해서 반환하되 -1을 곱함.
    return {'loss':-1 * np.mean(accuracy), 'status': STATUS_OK}

In [8]:
from hyperopt import fmin, tpe, Trials

trial_val = Trials()
best = fmin(fn=objective_func,
            space=xgb_search_space,
            algo=tpe.suggest,
            max_evals=50, # 최대 반복 횟수를 지정합니다.
            trials=trial_val, rstate=np.random.default_rng(seed=9))
print('best:', best)

100%|███████████████████████████████████████████████| 50/50 [00:12<00:00,  3.88trial/s, best loss: -0.9670616939700244]
best: {'colsample_bytree': 0.5424149213362504, 'learning_rate': 0.12601372924444681, 'max_depth': 17.0, 'min_child_weight': 2.0}


In [9]:
print('colsample_bytree:{0}, learning_rate:{1}, max_depth:{2}, min_child_weight:{3}'.format(
    round(best['colsample_bytree'], 5), round(best['learning_rate'], 5),
    int(best['max_depth']), int(best['min_child_weight'])))

# colsample_bytree: 각 트리를 구성하는데 사용될 열(특성)의 일부 비율
# learning_rate: 학습 속도 조정, 각 트리가 학습 데이터에 대해 얼마나 많이 기여할 지 결정
# max_depth: 트리의 최대 깊이를 제한하는 파라미터입니다. 
# 트리의 깊이가 깊어질수록 모델의 복잡도가 증가하므로, 과적합을 방지하기 위해 적절한 값을 선택해야 합니다.
# min_child_weight: 트리에서 각 리프 노드에 필요한 최소 가중치 합을 나타내는 파라미터입니다. 
# 이 값을 증가시키면 트리의 과적합을 방지할 수 있습니다.

colsample_bytree:0.54241, learning_rate:0.12601, max_depth:17, min_child_weight:2


In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

# 평가 : 사용자 함수로 
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [11]:
xgb_wrapper = XGBClassifier(n_estimators=400,
                            learning_rate=round(best['learning_rate'], 5),
                            max_depth=int(best['max_depth']),
                            min_child_weight=int(best['min_child_weight']),
                            colsample_bytree=round(best['colsample_bytree'], 5)
                           )

evals = [(X_tr, y_tr), (X_val, y_val)]
xgb_wrapper.fit(X_tr, y_tr, early_stopping_rounds=50, eval_metric='logloss',
                eval_set=evals, verbose=True)

preds = xgb_wrapper.predict(X_test)    # 학습한 거 가지고 실제 데이터를 넣어서 예측
pred_proba = xgb_wrapper.predict_proba(X_test)[:, 1]

get_clf_eval(y_test, preds, pred_proba)  

[0]	validation_0-logloss:0.58942	validation_1-logloss:0.62048
[1]	validation_0-logloss:0.50801	validation_1-logloss:0.55913
[2]	validation_0-logloss:0.44160	validation_1-logloss:0.50928
[3]	validation_0-logloss:0.38734	validation_1-logloss:0.46815
[4]	validation_0-logloss:0.34224	validation_1-logloss:0.43913
[5]	validation_0-logloss:0.30425	validation_1-logloss:0.41570
[6]	validation_0-logloss:0.27178	validation_1-logloss:0.38953
[7]	validation_0-logloss:0.24503	validation_1-logloss:0.37317
[8]	validation_0-logloss:0.22050	validation_1-logloss:0.35628
[9]	validation_0-logloss:0.19873	validation_1-logloss:0.33798
[10]	validation_0-logloss:0.17945	validation_1-logloss:0.32463
[11]	validation_0-logloss:0.16354	validation_1-logloss:0.31384
[12]	validation_0-logloss:0.15032	validation_1-logloss:0.30607
[13]	validation_0-logloss:0.13813	validation_1-logloss:0.30143
[14]	validation_0-logloss:0.12798	validation_1-logloss:0.29513
[15]	validation_0-logloss:0.11926	validation_1-logloss:0.28891
[1

[130]	validation_0-logloss:0.01639	validation_1-logloss:0.22934
[131]	validation_0-logloss:0.01634	validation_1-logloss:0.22987
[132]	validation_0-logloss:0.01629	validation_1-logloss:0.22927
[133]	validation_0-logloss:0.01624	validation_1-logloss:0.23076
[134]	validation_0-logloss:0.01620	validation_1-logloss:0.23030
[135]	validation_0-logloss:0.01615	validation_1-logloss:0.22891
[136]	validation_0-logloss:0.01610	validation_1-logloss:0.22883
[137]	validation_0-logloss:0.01606	validation_1-logloss:0.22882
[138]	validation_0-logloss:0.01601	validation_1-logloss:0.22876
[139]	validation_0-logloss:0.01597	validation_1-logloss:0.22734
[140]	validation_0-logloss:0.01592	validation_1-logloss:0.22882
[141]	validation_0-logloss:0.01588	validation_1-logloss:0.22935
[142]	validation_0-logloss:0.01583	validation_1-logloss:0.22880
[143]	validation_0-logloss:0.01579	validation_1-logloss:0.22856
[144]	validation_0-logloss:0.01575	validation_1-logloss:0.22725
[145]	validation_0-logloss:0.01571	valid

과제1_0524. 산탄데르 고객 만족 예측 모델을 생성하고 평가를 아래를 참조하여 수행하세요(XGBoost 모델 학습과 하이퍼 파라미터 튜닝)

- 데이터 읽어오기 : cust_df = pd.read_csv("./dataset/santander/train_santander.csv", encoding='latin-1')
- 데이터 전처리 : var3 피처 값 최빈값 2로 대체 및 ID 피처 드롭
- 데이터 분리 : 학습, 검증, 평가용
- 검색공간 설정
    - max_depth는 5에서 15까지 1간격으로,
    - min_child_weight는 1에서 6까지 1간격으로
    - colsample_bytree는 0.5에서 0.95사이,
    - learning_rate는 0.01에서 0.2사이 정규 분포된 값으로 설정
- 목적함수 설정
- 최적 하이퍼파라미터 산출
- 최적 하이퍼파라미터 기반 학습 및 예측
- 평가(ROC AUC)
- 변수 중요도 시각화

Kaggle Competitions<br>
Santander Customer Satisfaction<br>
Which customers are happy customers?

일선 지원 팀에서 최고 경영진에 이르기까지 고객 만족은 성공의 핵심 척도입니다. 만족도가 낮은 고객은 은행의 서비스를 더이상 이용하지 않습니다. 더나가 불만족 고객은 서비스 이용을 중단하기 전에 불만을 표현하는 경우가 거의 없습니다.

Santander Bank는 Kagglers에게 서비스 초기에 불만족한 고객을 식별할 수 있도록 도와달라고 요청하고 있습니다. 그렇게 하면 Santander는 시기를 놓치지 않고 고객의 행복을 개선하기 위한 사전 조치를 취할 수 있습니다.

이 대회에서는 수백 가지의 익명화된 기능을 사용하여 고객이 은행 업무 경험에 만족하는지 불만족하는지 예측하게 됩니다.

제출물은 예측 확률과 관찰 대상 사이의 ROC 곡선 아래 영역에서 평가됩니다.

상금 : U$ 60,000

https://www.kaggle.com/c/santander-customer-satisfaction

# 데이터 가져오기

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("./dataset/train_santander.csv", encoding = 'latin-1')
print('dataset shape:', df.shape)
df.head(3)

dataset shape: (76020, 371)


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.17,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.03,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.77,0


In [2]:
# label(TARGET)이 0이면 만족한 고객, 1이면 불만을 가진 고객

print(df['TARGET'].value_counts())
unsatisfied_count = df[df['TARGET'] == 1].TARGET.count()
total_count = df.TARGET.count()
print('unsatisfied 비율은 {0:.2f}'.format((unsatisfied_count / total_count)))

0    73012
1     3008
Name: TARGET, dtype: int64
unsatisfied 비율은 0.04


# 데이터 전처리
:데이터 전처리 : var3 피처 값 최빈값 2로 대체 및 ID 피처 드롭

In [3]:
df.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,1.000000,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,38104.750000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,76043.000000,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,113748.750000,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,151838.000000,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


In [4]:
df.var3.value_counts()

 2         74165
 8           138
-999999      116
 9           110
 3           108
           ...  
 231           1
 188           1
 168           1
 135           1
 87            1
Name: var3, Length: 208, dtype: int64

In [5]:
# 데이터 전처리 : var3 피처 값 최빈값 2로 대체 및 ID 피처 드롭

df.var3.replace(-999999, 2, inplace = True)
df.var3.value_counts()

2      74281
8        138
9        110
3        108
1        105
       ...  
231        1
188        1
168        1
135        1
87         1
Name: var3, Length: 207, dtype: int64

In [7]:
# ID 피처 드롭
df.drop('ID', axis=1, inplace = True)
df.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [9]:
# 피처 세트와 레이블 세트 분리. 레이블 컬럼은 DataFrame 의 맨 마지막에 위치해 컬럼 위치 -1로 분리

X_features = df.iloc[:, :-1]
y_labels = df.iloc[: , -1]
print('피처 데이터 shape: {0}'. format(X_features.shape))

피처 데이터 shape: (76020, 369)


# 데이터 분리 : 학습, 검증, 평가용

In [10]:
# 원본 데이터 세트에서 학습 데이터 세트와 테스트 데이터 세트 분리

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels,
                                                   test_size = 0.2, random_state = 0)

# 비중 보기
train_count =  y_train.count()
test_count = y_test.count()
print('학습 세트 Shape:{0}, 테스트 세트 Shape: {1}'. format(X_train.shape, X_test.shape))

print('학습 세트 레이블 값 분포 비율')
print(y_train.value_counts()/train_count)
print('\n 테스트 세트 레이블 값 분포 비율')
print(y_test.value_counts()/test_count)

학습 세트 Shape:(60816, 369), 테스트 세트 Shape: (15204, 369)
학습 세트 레이블 값 분포 비율
0    0.960964
1    0.039036
Name: TARGET, dtype: float64

 테스트 세트 레이블 값 분포 비율
0    0.9583
1    0.0417
Name: TARGET, dtype: float64


In [11]:
# X_train, y_train을 다시 학습과 검증 데이터 세트로 분리
X_tr,X_val, y_tr, y_val = train_test_split(X_train, y_train,
                                                  test_size=0.3, random_state=0)

### XGBoost 모델 학습과 하이퍼 파라미터 튜닝

In [34]:
# 검사 전 한번 평가 해보겠음

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# n_estimators는 500으로 설정하고, learning_rate는 0.05로 설정하며, random_state는 일관된 예측 결과를 위해 156으로 설정합니다.
xgb_clf = XGBClassifier(n_estimators=500, learning_rate=0.05, random_state=156)

# 성능 평가 지표로 AUC를 사용하고, 조기 중단을 위한 라운드는 100으로 설정하여 학습을 수행합니다.
xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=100, eval_metric='auc', eval_set=[(X_tr, y_tr), (X_val, y_val)])

xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

[0]	validation_0-auc:0.82179	validation_1-auc:0.80068
[1]	validation_0-auc:0.82347	validation_1-auc:0.80523
[2]	validation_0-auc:0.83178	validation_1-auc:0.81097
[3]	validation_0-auc:0.83401	validation_1-auc:0.81091
[4]	validation_0-auc:0.83443	validation_1-auc:0.81040
[5]	validation_0-auc:0.83570	validation_1-auc:0.81089
[6]	validation_0-auc:0.83597	validation_1-auc:0.81057
[7]	validation_0-auc:0.83643	validation_1-auc:0.81082
[8]	validation_0-auc:0.83682	validation_1-auc:0.81147
[9]	validation_0-auc:0.83769	validation_1-auc:0.81188
[10]	validation_0-auc:0.83770	validation_1-auc:0.81163
[11]	validation_0-auc:0.83911	validation_1-auc:0.81355
[12]	validation_0-auc:0.83976	validation_1-auc:0.81336
[13]	validation_0-auc:0.84038	validation_1-auc:0.81365
[14]	validation_0-auc:0.84176	validation_1-auc:0.81419
[15]	validation_0-auc:0.84306	validation_1-auc:0.81586
[16]	validation_0-auc:0.84343	validation_1-auc:0.81610
[17]	validation_0-auc:0.84373	validation_1-auc:0.81584
[18]	validation_0-au

[149]	validation_0-auc:0.90500	validation_1-auc:0.83353
[150]	validation_0-auc:0.90513	validation_1-auc:0.83352
[151]	validation_0-auc:0.90531	validation_1-auc:0.83349
[152]	validation_0-auc:0.90548	validation_1-auc:0.83349
[153]	validation_0-auc:0.90551	validation_1-auc:0.83351
[154]	validation_0-auc:0.90563	validation_1-auc:0.83348
[155]	validation_0-auc:0.90572	validation_1-auc:0.83343
[156]	validation_0-auc:0.90579	validation_1-auc:0.83344
[157]	validation_0-auc:0.90591	validation_1-auc:0.83345
[158]	validation_0-auc:0.90621	validation_1-auc:0.83355
[159]	validation_0-auc:0.90630	validation_1-auc:0.83359
[160]	validation_0-auc:0.90643	validation_1-auc:0.83356
[161]	validation_0-auc:0.90657	validation_1-auc:0.83352
[162]	validation_0-auc:0.90672	validation_1-auc:0.83346
[163]	validation_0-auc:0.90694	validation_1-auc:0.83347
[164]	validation_0-auc:0.90714	validation_1-auc:0.83343
[165]	validation_0-auc:0.90720	validation_1-auc:0.83343
[166]	validation_0-auc:0.90727	validation_1-auc:

### 검색공간 설정
- max_depth는 5에서 15까지 1간격으로,
- min_child_weight는 1에서 6까지 1간격으로
- colsample_bytree는 0.5에서 0.95사이,
- learning_rate는 0.01에서 0.2사이 정규 분포된 값으로 설정

In [50]:
from hyperopt import hp

# max_depth는 5에서 15까지 1간격으로, minchild_weight는 1에서 6까지 1간격으로
# colsample_bytree는 0.5에서 0.95사이, learning_rate는 0.01에서 0.2사이 정규 분포된 값으로 검색.

xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 15, 1),
                    'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1), 
                    'colsample_bytree': hp. uniform ('colsample_bytree', 0.5, 0.95), 
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2)
}

### 목적함수 설정

In [51]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

# 목적 함수 설정. 
# 추후 fmin()에서 입력된 search_space값으로 XGBClassifier 교차 검증 학습 후 -1* roc_auc 평균 값을 반환.  
def objective_func(search_space):
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            colsample_bytree=search_space['colsample_bytree'],
                            learning_rate=search_space['learning_rate']
                           )
    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list= []
    
    # 3개 k-fold방식 적용 
    kf = KFold(n_splits=3)
    
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train):
        
        # kf.split(X_train)으로 추출된 학습과 검증 index값으로 학습과 검증 데이터 세트 분리 
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]
        
        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행. 
        xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric='auc',
                   eval_set=[(X_tr, y_tr), (X_val, y_val)])
    
        # 1로 예측한 확률값 추출후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결과값 담음. 
        score = roc_auc_score(y_val, xgb_clf.predict_proba(X_val)[:, 1])
        roc_auc_list.append(score)
        
    # 3개 k-fold로 계산된 roc_auc값의 평균값을 반환하되, 
    # HyperOpt는 목적함수의 최소값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환. 
    return -1 * np.mean(roc_auc_list)


### 최적 하이퍼파라미터 산출

In [56]:
from hyperopt import fmin, tpe, Trials

trials = Trials()

# fmin()함수를 호출. max_evals지정된 횟수만큼 반복 후 목적함수의 최소값을 가지는 최적 입력값 추출.
best = fmin(fn=objective_func,
            space=xgb_search_space,
            algo=tpe.suggest,
            max_evals=50, # 최대 반복 횟수를 지정합니다.
            trials=trials, rstate=np.random.default_rng(seed=30))

print('best:', best)

[0]	validation_0-auc:0.81678	validation_1-auc:0.79160                                                                  
[1]	validation_0-auc:0.82454	validation_1-auc:0.79688                                                                  
[2]	validation_0-auc:0.83323	validation_1-auc:0.80572                                                                  
[3]	validation_0-auc:0.83854	validation_1-auc:0.81095                                                                  
[4]	validation_0-auc:0.83847	validation_1-auc:0.80989                                                                  
[5]	validation_0-auc:0.83879	validation_1-auc:0.80978                                                                  
[6]	validation_0-auc:0.84053	validation_1-auc:0.81042                                                                  
[7]	validation_0-auc:0.84129	validation_1-auc:0.81116                                                                  
[8]	validation_0-auc:0.84224	validation_

[68]	validation_0-auc:0.89250	validation_1-auc:0.83282                                                                 
[69]	validation_0-auc:0.89269	validation_1-auc:0.83292                                                                 
[70]	validation_0-auc:0.89308	validation_1-auc:0.83292                                                                 
[71]	validation_0-auc:0.89361	validation_1-auc:0.83264                                                                 
[72]	validation_0-auc:0.89406	validation_1-auc:0.83256                                                                 
[73]	validation_0-auc:0.89430	validation_1-auc:0.83239                                                                 
[74]	validation_0-auc:0.89457	validation_1-auc:0.83261                                                                 
[75]	validation_0-auc:0.89482	validation_1-auc:0.83233                                                                 
[76]	validation_0-auc:0.89494	validation

[47]	validation_0-auc:0.88318	validation_1-auc:0.83603                                                                 
[48]	validation_0-auc:0.88406	validation_1-auc:0.83587                                                                 
[49]	validation_0-auc:0.88499	validation_1-auc:0.83567                                                                 
[50]	validation_0-auc:0.88567	validation_1-auc:0.83621                                                                 
[51]	validation_0-auc:0.88617	validation_1-auc:0.83640                                                                 
[52]	validation_0-auc:0.88688	validation_1-auc:0.83592                                                                 
[53]	validation_0-auc:0.88747	validation_1-auc:0.83641                                                                 
[54]	validation_0-auc:0.88846	validation_1-auc:0.83647                                                                 
[55]	validation_0-auc:0.88912	validation

[15]	validation_0-auc:0.85543	validation_1-auc:0.82596                                                                 
[16]	validation_0-auc:0.85660	validation_1-auc:0.82658                                                                 
[17]	validation_0-auc:0.85740	validation_1-auc:0.82710                                                                 
[18]	validation_0-auc:0.85886	validation_1-auc:0.82679                                                                 
[19]	validation_0-auc:0.85976	validation_1-auc:0.82722                                                                 
[20]	validation_0-auc:0.86042	validation_1-auc:0.82733                                                                 
[21]	validation_0-auc:0.86157	validation_1-auc:0.82861                                                                 
[22]	validation_0-auc:0.86180	validation_1-auc:0.82913                                                                 
[23]	validation_0-auc:0.86289	validation

[83]	validation_0-auc:0.90074	validation_1-auc:0.83709                                                                 
[84]	validation_0-auc:0.90102	validation_1-auc:0.83728                                                                 
[85]	validation_0-auc:0.90138	validation_1-auc:0.83751                                                                 
[86]	validation_0-auc:0.90155	validation_1-auc:0.83765                                                                 
[87]	validation_0-auc:0.90164	validation_1-auc:0.83763                                                                 
[88]	validation_0-auc:0.90170	validation_1-auc:0.83761                                                                 
[89]	validation_0-auc:0.90205	validation_1-auc:0.83762                                                                 
[90]	validation_0-auc:0.90245	validation_1-auc:0.83791                                                                 
[91]	validation_0-auc:0.90253	validation

...

[42]	validation_0-auc:0.93123	validation_1-auc:0.83225                                                                 
[43]	validation_0-auc:0.93244	validation_1-auc:0.83169                                                                 
[44]	validation_0-auc:0.93272	validation_1-auc:0.83132                                                                 
[45]	validation_0-auc:0.93310	validation_1-auc:0.83115                                                                 
[46]	validation_0-auc:0.93327	validation_1-auc:0.83107                                                                 
[47]	validation_0-auc:0.93385	validation_1-auc:0.83083                                                                 
[48]	validation_0-auc:0.93411	validation_1-auc:0.83060                                                                 
[49]	validation_0-auc:0.93430	validation_1-auc:0.83065                                                                 
[0]	validation_0-auc:0.83147	validation_

[60]	validation_0-auc:0.94143	validation_1-auc:0.82720                                                                 
[61]	validation_0-auc:0.94175	validation_1-auc:0.82702                                                                 
[62]	validation_0-auc:0.94218	validation_1-auc:0.82687                                                                 
[63]	validation_0-auc:0.94254	validation_1-auc:0.82639                                                                 
[64]	validation_0-auc:0.94283	validation_1-auc:0.82657                                                                 
[0]	validation_0-auc:0.82765	validation_1-auc:0.80030                                                                  
[1]	validation_0-auc:0.83571	validation_1-auc:0.80383                                                                  
[2]	validation_0-auc:0.84131	validation_1-auc:0.80740                                                                  
[3]	validation_0-auc:0.84543	validation_

[63]	validation_0-auc:0.91909	validation_1-auc:0.83029                                                                 
[64]	validation_0-auc:0.91935	validation_1-auc:0.83003                                                                 
[0]	validation_0-auc:0.82208	validation_1-auc:0.80473                                                                  
[1]	validation_0-auc:0.83174	validation_1-auc:0.81745                                                                  
[2]	validation_0-auc:0.83931	validation_1-auc:0.82230                                                                  
[3]	validation_0-auc:0.84601	validation_1-auc:0.82363                                                                  
[4]	validation_0-auc:0.84829	validation_1-auc:0.82400                                                                  
[5]	validation_0-auc:0.84961	validation_1-auc:0.82427                                                                  
[6]	validation_0-auc:0.85279	validation_

[66]	validation_0-auc:0.92124	validation_1-auc:0.83552                                                                 
[67]	validation_0-auc:0.92190	validation_1-auc:0.83529                                                                 
[68]	validation_0-auc:0.92221	validation_1-auc:0.83539                                                                 
[69]	validation_0-auc:0.92258	validation_1-auc:0.83547                                                                 
[70]	validation_0-auc:0.92261	validation_1-auc:0.83549                                                                 
[71]	validation_0-auc:0.92275	validation_1-auc:0.83553                                                                 
[72]	validation_0-auc:0.92326	validation_1-auc:0.83523                                                                 
[73]	validation_0-auc:0.92364	validation_1-auc:0.83524                                                                 
[74]	validation_0-auc:0.92374	validation

[56]	validation_0-auc:0.91816	validation_1-auc:0.83524                                                                 
[57]	validation_0-auc:0.91895	validation_1-auc:0.83493                                                                 
[58]	validation_0-auc:0.91935	validation_1-auc:0.83494                                                                 
[59]	validation_0-auc:0.91977	validation_1-auc:0.83483                                                                 
[60]	validation_0-auc:0.92023	validation_1-auc:0.83458                                                                 
[61]	validation_0-auc:0.92097	validation_1-auc:0.83453                                                                 
[62]	validation_0-auc:0.92154	validation_1-auc:0.83450                                                                 
[63]	validation_0-auc:0.92177	validation_1-auc:0.83435                                                                 
[64]	validation_0-auc:0.92193	validation

### 최적 하이퍼파라미터 기반 학습 및 예측 / 평가(ROC AUC)

In [53]:
# n_estimators를 500증가 후 최적으로 찾은 하이퍼 파라미터를 기반으로 학습과 예측 수행.
xgb_clf = XGBClassifier(n_estimators=500, learning_rate=round(best['learning_rate'], 5),
                        max_depth=int(best['max_depth']), min_child_weight=int(best['min_child_weight']), 
                        colsample_bytree=round(best['colsample_bytree'], 5)   
                       )

# evaluation metric을 auc로, early stopping은 100 으로 설정하고 학습 수행. 
xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=100, 
            eval_metric="auc",eval_set=[(X_tr, y_tr), (X_val, y_val)])

xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1])
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

[0]	validation_0-auc:0.81902	validation_1-auc:0.80646
[1]	validation_0-auc:0.82802	validation_1-auc:0.81008
[2]	validation_0-auc:0.83126	validation_1-auc:0.81286
[3]	validation_0-auc:0.83239	validation_1-auc:0.81221
[4]	validation_0-auc:0.83608	validation_1-auc:0.81141
[5]	validation_0-auc:0.84103	validation_1-auc:0.81686
[6]	validation_0-auc:0.84462	validation_1-auc:0.81850
[7]	validation_0-auc:0.84498	validation_1-auc:0.81835
[8]	validation_0-auc:0.84650	validation_1-auc:0.81916
[9]	validation_0-auc:0.84669	validation_1-auc:0.81866
[10]	validation_0-auc:0.84426	validation_1-auc:0.81671
[11]	validation_0-auc:0.84402	validation_1-auc:0.81706
[12]	validation_0-auc:0.84707	validation_1-auc:0.81956
[13]	validation_0-auc:0.85083	validation_1-auc:0.82234
[14]	validation_0-auc:0.85245	validation_1-auc:0.82425
[15]	validation_0-auc:0.85463	validation_1-auc:0.82629
[16]	validation_0-auc:0.85502	validation_1-auc:0.82669
[17]	validation_0-auc:0.85736	validation_1-auc:0.82757
[18]	validation_0-au

[149]	validation_0-auc:0.90590	validation_1-auc:0.83480
[150]	validation_0-auc:0.90622	validation_1-auc:0.83468
[151]	validation_0-auc:0.90626	validation_1-auc:0.83471
[152]	validation_0-auc:0.90640	validation_1-auc:0.83470
[153]	validation_0-auc:0.90653	validation_1-auc:0.83474
[154]	validation_0-auc:0.90684	validation_1-auc:0.83477
[155]	validation_0-auc:0.90699	validation_1-auc:0.83469
[156]	validation_0-auc:0.90714	validation_1-auc:0.83481
[157]	validation_0-auc:0.90721	validation_1-auc:0.83473
[158]	validation_0-auc:0.90735	validation_1-auc:0.83453
[159]	validation_0-auc:0.90763	validation_1-auc:0.83426
[160]	validation_0-auc:0.90779	validation_1-auc:0.83417
[161]	validation_0-auc:0.90813	validation_1-auc:0.83406
[162]	validation_0-auc:0.90814	validation_1-auc:0.83398
[163]	validation_0-auc:0.90819	validation_1-auc:0.83399
[164]	validation_0-auc:0.90846	validation_1-auc:0.83388
[165]	validation_0-auc:0.90861	validation_1-auc:0.83381
[166]	validation_0-auc:0.90866	validation_1-auc:

### 변수 중요도 시각화

In [ ]:
# 이를 시각화

from xgboost import plot_importance
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplot(1,1, figsize=(10,8))    # 1개의 그래프를 그릴거니까 1,1
plot_importance(xgb_clf, ax=ax, max_num_features=20, height=0.4)

# LightGBM 모델 학습과 하이퍼 파라미터 튜닝

과제1_0525. 산탄데르 고객 만족 예측을 lightgbm을 이용해서 수행하세요. (HyperOPT를 이용한 하이퍼 파라미터 튜닝)

In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=500)

eval_set = [(X_tr, y_tr), (X_val, y_val)]
lgbm_clf.fit(X_tr, y_tr, early_stopping_rounds=100, eval_metric = "auc", eval_set=eval_set)

lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predicct_proba(X_test)[:,1])
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))